## This Notebook is a first part of Research to DFDC


Advanced knowledge in probability theory, numerical methods will not give you much boost in your prediction ;)

Since I have a weak computer with only 8 GB of RAM and don't have a GPU, also Kaggle limits me to 30 hours of weekly use, I don’t have much opportunity to create a good predictor that can learn 500 GB of data :)).

And so I decided to do the approximation of the outputs to real labels, and here is what came of it:

I was able to squeeze 0.41423 from the ensemble of two architectures ResNext and XCeption

In [ ]:
import os

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)

from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
%matplotlib inline

In [ ]:
TestVideosDirectory = "test_videos/"

TestVideos = sorted([x for x in os.listdir(TestVideosDirectory) if x[-4:] == ".mp4"])

Metadata = pd.read_csv('data/metadata')

Metadata

In [ ]:
TestVideoFilter = Metadata.filename.isin(TestVideos)

TestData = Metadata[TestVideoFilter]

TestData.sort_values(by = 'filename', inplace = True)

TestData.reset_index(drop = True, inplace = True)

TestData

In [ ]:
Submission = pd.read_csv('data/submission-4.csv')

Submission046 = pd.read_csv('data/sub0467x.csv')

Submission['rlabel'] = TestData.label

Submission.rlabel = Submission.rlabel.map({'FAKE': 1, 'REAL': 0})

Submission.info()

print(log_loss(Submission.rlabel, Submission.label))

In [ ]:
sns.distplot(Submission.rlabel)

sns.distplot(Submission.label)

In [ ]:
sns.distplot(Submission.label)

sns.distplot(Submission046.label)

plt.legend(labels = ['better','046'])

## Logistic Regression

Just her coefs gave boost

In [ ]:
from sklearn.linear_model import LogisticRegression as LR

SubmissionWithCalibration = Submission.copy()

x = np.array(Submission.label)

lr = LR(penalty = 'l2', dual = True, tol = 0.0001, C = 1.3, fit_intercept = True, intercept_scaling = 1.0, 
        
        class_weight = None, random_state = None)     

lr.fit(x.reshape(-1, 1), Submission.rlabel)   

p_calibrated = lr.predict_proba(x.reshape(-1, 1))[:,1]

print(lr.coef_)

print(lr.intercept_)

print(log_loss(SubmissionWithCalibration.rlabel, p_calibrated))

In [ ]:
sns.distplot(Submission.rlabel)

sns.distplot(p_calibrated)

# Isotonic Regression

Waste of time

In [ ]:
from sklearn.isotonic import IsotonicRegression as IR

ir = IR() # out_of_bounds = 'clip'

ir.fit(Submission.label, Submission.rlabel)

IRp_calibrated = ir.transform(Submission.label)

In [ ]:
sns.distplot(Submission.rlabel)

sns.distplot(IRp_calibrated)

# Gradient Boosting Trees

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators = 300)

xgb.fit(x.reshape(-1, 1), Submission.rlabel)

XGBCalibrated = xgb.predict_proba(x.reshape(-1, 1))[:,1]

In [ ]:
sns.distplot(Submission.rlabel)

sns.distplot(XGBCalibrated)

In [ ]:
SubmissionWithCalibration['IRCalibration'] = IRp_calibrated

SubmissionWithCalibration['LRCalibration'] = p_calibrated

SubmissionWithCalibration['XGBCalibration'] =  XGBCalibrated

SubmissionWithCalibration

In [ ]:
print("Best submission on kaggle ", log_loss(SubmissionWithCalibration.rlabel, SubmissionWithCalibration.label))

print("Isotonic Regression calibration", log_loss(SubmissionWithCalibration.rlabel, SubmissionWithCalibration.IRCalibration))

print("Logistic Regression calibration", log_loss(SubmissionWithCalibration.rlabel, SubmissionWithCalibration.LRCalibration))

print("XGBoost calibration", log_loss(SubmissionWithCalibration.rlabel, SubmissionWithCalibration.XGBCalibration))